<a href="https://colab.research.google.com/github/Aspartamei/Projekt2022_UMwF/blob/main/Projekt2022_UMwF_Przygotowanie_danych.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install yfinance
import pandas as pd
import yfinance as yf
import numpy as np
import math
import pickle
import time
import datetime
from datetime import date, timedelta, datetime
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.2 MB/s 
     |████████████████████████████████| 6.4 MB 8.3 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Wczytanie danych bazowych


In [5]:
url = "https://raw.githubusercontent.com/Aspartamei/Projekt2022_UMwF/main/DANE1.csv"
df = pd.read_csv(url, sep=',', header = None, names = ['Date','Comp2','Comp','Category','Code','Score'])
df = df.drop('Code', axis = 1)
df = df.drop('Comp2', axis = 1)
df = df.drop('Category', axis = 1)
df = df.reset_index(drop = True)
df["Date"] = pd.to_datetime(df["Date"])

In [7]:
df.head()


,Date,Comp,Score
0,2004-02-11,SU,0.953727
1,2004-02-11,GGG,0.952753
2,2004-02-11,WGR,0.947634
3,2004-02-11,CWT,0.934181
4,2004-02-11,BLL,0.922862


In [ ]:
df.shape

(37360, 3)

Pobranie cen zamknięcia(close) z Yahoo Finance dla każdej firmy(comp)

In [ ]:
names = set(df.Comp)
beginnig_date = datetime(2004, 2, 11)
end_date = datetime(2022, 2, 9)

In [ ]:
close_price_df = pd.DataFrame()

for comp in names:
    getInfo = yf.Ticker(comp)
    getHistory = getInfo.history(getInfo, start=beginnig_date, end=end_date)

    getHistory["Comp"] = comp
    getHistory["Date"] = getHistory.index
    close_price_df = close_price_df.append(getHistory, ignore_index=True)

close_price_df = close_price_df.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits", "Adj Close"])

- CBM: No data found, symbol may be delisted
- GAS.XX1: No data found, symbol may be delisted
- MBG.XX9: No data found, symbol may be delisted
- NSR.XX2: No data found, symbol may be delisted
- CATM: No data found, symbol may be delisted
- TIBX: No data found for this date range, symbol may be delisted
- XLNX: No data found, symbol may be delisted
- WLP.XX9: No data found, symbol may be delisted
- RSE.XX1: No data found, symbol may be delisted
- DCP.XX1: No data found, symbol may be delisted
- TLM: No data found for this date range, symbol may be delisted
- WFSI: No data found for this date range, symbol may be delisted
- SUN.XX1: No data found, symbol may be delisted
- BJ.XX10: No data found, symbol may be delisted
- GYI: No data found for this date range, symbol may be delisted
- KNL: No data found, symbol may be delisted
- KWKAQ: No data found for this date range, symbol may be delisted
- AGE.XX2: No data found, symbol may be delisted
- UST.XX1: No data found, symbol may be delisted

In [ ]:
close_price_df.shape

(5520441, 3)

In [ ]:
close_price_df.head()

,Close,Comp,Date
0,6.699813,ANDV,2004-02-11
1,6.841122,ANDV,2004-02-12
2,6.662407,ANDV,2004-02-13
3,6.945029,ANDV,2004-02-17
4,7.148684,ANDV,2004-02-18


Dodanie kolumny Close(cena zamkniecia) poprzez połączenie zbiorów

In [ ]:
close_score_df = pd.merge(df, close_price_df[["Close", "Comp", "Date"]], how="left", on=["Comp", "Date"])
close_score_df

,Date,Comp,Score,Close
0,2004-02-11,SU,0.953727,9.165979
1,2004-02-11,GGG,0.952753,6.536162
2,2004-02-11,WGR,0.947634,NaN
3,2004-02-11,CWT,0.934181,9.002888
4,2004-02-11,BLL,0.922862,6.879496
...,...,...,...,...
37355,2022-02-09,PEP,0.701507,NaN
37356,2022-02-09,SSNC,0.701123,NaN
37357,2022-02-09,GEF,0.697954,NaN
37358,2022-02-09,DPZ,0.697741,NaN


Wyznaczenie stóp zwrotu 

In [ ]:
close_score_df['Date'] = pd.to_datetime(close_score_df['Date'])

In [ ]:
close_score_df

,Date,Comp,Score,Close
0,2004-02-11,SU,0.953727,9.165979
1,2004-02-11,GGG,0.952753,6.536162
2,2004-02-11,WGR,0.947634,NaN
3,2004-02-11,CWT,0.934181,9.002888
4,2004-02-11,BLL,0.922862,6.879496
...,...,...,...,...
37355,2022-02-09,PEP,0.701507,NaN
37356,2022-02-09,SSNC,0.701123,NaN
37357,2022-02-09,GEF,0.697954,NaN
37358,2022-02-09,DPZ,0.697741,NaN


In [ ]:
close_score_df = close_score_df[["Close", "Date", "Comp","Score"]]

In [ ]:
close_score_df

,Close,Date,Comp,Score
0,9.165984,2004-02-11,SU,0.953727
1,6.536161,2004-02-11,GGG,0.952753
2,NaN,2004-02-11,WGR,0.947634
3,9.002892,2004-02-11,CWT,0.934181
4,6.879497,2004-02-11,BLL,0.922862
...,...,...,...,...
37355,NaN,2022-02-09,PEP,0.701507
37356,NaN,2022-02-09,SSNC,0.701123
37357,NaN,2022-02-09,GEF,0.697954
37358,NaN,2022-02-09,DPZ,0.697741


Obliczenie stóp zwrotu w horyzontach: miesięcznym(month), kwartalnym(qtr) i roczny(year)

In [ ]:
company = set(close_price_df.Comp)
all_ror_df = pd.DataFrame()
for comp in company:
    company_df = close_price_df[close_price_df.Comp == comp].copy()
    company_df['Date'] = pd.to_datetime(company_df['Date'])
    for date in company_df.Date:
        try:
            c_t1 = company_df[company_df.Date == date].iloc[0, 0]

            # Month Value
            date_plus_month = date + pd.DateOffset(months=1)
            if date_plus_month in set(company_df.Date):
                c_t_month = company_df[company_df.Date == date_plus_month].iloc[0, 0]
                month_ror = math.log(c_t_month / c_t1)
                company_df.loc[company_df["Date"] == date_plus_month, "month_ROR"] = month_ror
        
        except ValueError:
            month_ror = np.nan
            continue
            
        try:
            # Qtr Value
            date_plus_qtr = date + pd.DateOffset(months=3)
            if date_plus_qtr in set(company_df.Date): 
                c_t_qtr = company_df[company_df.Date == date_plus_qtr].iloc[0, 0]
                qtr_ror = math.log(c_t_qtr / c_t1)
                company_df.loc[company_df["Date"] == date_plus_qtr, "qtr_ROR"] = qtr_ror
                
        except ValueError:
            qtr_ror = np.nan
            continue

        try:     
            # Year Value
            date_plus_year = date + pd.DateOffset(years=1)
            if date_plus_year in set(company_df.Date):
                c_t_year = company_df[company_df.Date == date_plus_year].iloc[0, 0]
                year_ror = math.log(c_t_year / c_t1)
                company_df.loc[company_df["Date"] == date_plus_year, "year_ROR"] = year_ror
            
        except ValueError:
            year_ror = np.nan
            continue
    all_ror_df = all_ror_df.append(company_df, ignore_index=True)

Połączenie zbiorów i usunięcie braków danych. 
Dane dla niektórych spółek nie pobrały się z Yahoo Finance a Score ma inny interwał niż zmienna Close. 

In [24]:
ROR_df = pd.merge(all_ror_df, df[["Comp", "Date", "Score"]], how="left", on=["Comp", "Date"])
ROR_df

,Comp,Date,Close,month_ROR,qtr_ROR,year_ROR,Score
0,UBA,2004-03-24,6.163362,0.083411,NaN,NaN,0.741122
1,UBA,2005-06-15,7.158419,0.087692,0.098211,0.254842,0.803913
2,UBA,2005-06-29,7.281594,0.045381,0.134050,0.223035,0.806620
3,UBA,2005-07-13,7.605220,0.066313,0.228741,0.219617,0.923381
4,UBA,2005-07-27,7.613519,0.062501,0.249711,0.305484,0.923270
...,...,...,...,...,...,...,...
30102,AMED,2010-02-10,56.180000,0.054901,0.305918,0.206022,0.485908
30103,AMED,2017-07-12,58.970001,-0.002406,0.152455,0.160072,0.689752
30104,AMED,2017-07-26,48.689999,-0.120590,0.079048,0.075266,0.677474
30105,AMED,2019-03-13,122.250000,-0.113183,-0.090947,0.683718,0.751574


In [25]:

ROR_df = ROR_df[["Comp", "Date", "Close","month_ROR","qtr_ROR","year_ROR","Score"]]
no_score = []
for comp in ROR_df.Comp.unique():
    comp_df = ROR_df[ROR_df.Comp == comp]
    if comp_df.Score.isna().sum() == len(comp_df.Score):
        no_score.append(comp)
data_clean_df = pd.DataFrame()
data_clean_df = ROR_df[~ROR_df['Comp'].isin(no_score)]


In [26]:
data_clean_df

,Comp,Date,Close,month_ROR,qtr_ROR,year_ROR,Score
0,UBA,2004-03-24,6.163362,0.083411,NaN,NaN,0.741122
1,UBA,2005-06-15,7.158419,0.087692,0.098211,0.254842,0.803913
2,UBA,2005-06-29,7.281594,0.045381,0.134050,0.223035,0.806620
3,UBA,2005-07-13,7.605220,0.066313,0.228741,0.219617,0.923381
4,UBA,2005-07-27,7.613519,0.062501,0.249711,0.305484,0.923270
...,...,...,...,...,...,...,...
30102,AMED,2010-02-10,56.180000,0.054901,0.305918,0.206022,0.485908
30103,AMED,2017-07-12,58.970001,-0.002406,0.152455,0.160072,0.689752
30104,AMED,2017-07-26,48.689999,-0.120590,0.079048,0.075266,0.677474
30105,AMED,2019-03-13,122.250000,-0.113183,-0.090947,0.683718,0.751574


Obliczenie średnich stóp zwrotu

In [29]:
mean_ror_df = pd.DataFrame()
for comp in tqdm(data_clean_df.Comp.unique()):
    mean_df = pd.DataFrame()
    mean_df = data_clean_df[data_clean_df.Comp == comp].copy()
    score_index = mean_df[~mean_df.Score.isna()].index
    len(score_index)
    for indx in score_index:
        start_date = mean_df.loc[indx, 'Date']
        dates_indx = mean_df.loc[(mean_df.Date >= (start_date - timedelta(days = 7))) & (mean_df.Date <= start_date)].index
        mean_df.loc[indx, 'month_ROR'] = mean_df.loc[dates_indx ,'month_ROR'].mean()
        mean_df.loc[indx, 'qtr_ROR'] = mean_df.loc[dates_indx ,'qtr_ROR'].mean()
        mean_df.loc[indx, 'year_ROR'] = mean_df.loc[dates_indx ,'year_ROR'].mean()
    
    mean_ror_df = pd.concat([mean_ror_df, mean_df[mean_df.index.isin(score_index)]])
mean_ror_df = mean_ror_df.reset_index(drop = True)
print(mean_ror_df)

100%|██████████| 1352/1352 [01:45<00:00, 12.77it/s]

       Comp       Date       Close  month_ROR   qtr_ROR  year_ROR     Score
0       UBA 2004-03-24    6.163362   0.083411       NaN       NaN  0.741122
1       UBA 2005-06-15    7.158419   0.087692  0.098211  0.254842  0.803913
2       UBA 2005-06-29    7.281594   0.045381  0.134050  0.223035  0.806620
3       UBA 2005-07-13    7.605220   0.066313  0.228741  0.219617  0.923381
4       UBA 2005-07-27    7.613519   0.062501  0.249711  0.305484  0.923270
...     ...        ...         ...        ...       ...       ...       ...
30102  AMED 2010-02-10   56.180000   0.054901  0.305918  0.206022  0.485908
30103  AMED 2017-07-12   58.970001  -0.002406  0.152455  0.160072  0.689752
30104  AMED 2017-07-26   48.689999  -0.120590  0.079048  0.075266  0.677474
30105  AMED 2019-03-13  122.250000  -0.113183 -0.090947  0.683718  0.751574
30106  AMED 2019-03-27  119.559998  -0.024715  0.099533  0.695205  0.751194

[30107 rows x 7 columns]


In [ ]:
mean_ror_df

,Comp,Date,Close,month_ROR,qtr_ROR,year_ROR,Score
0,UBA,2004-03-24,6.163362,0.083411,NaN,NaN,0.741122
1,UBA,2005-06-15,7.158419,0.087692,0.098211,0.254842,0.803913
2,UBA,2005-06-29,7.281594,0.045381,0.134050,0.223035,0.806620
3,UBA,2005-07-13,7.605220,0.066313,0.228741,0.219617,0.923381
4,UBA,2005-07-27,7.613519,0.062501,0.249711,0.305484,0.923270
...,...,...,...,...,...,...,...
30102,AMED,2010-02-10,56.180000,0.054901,0.305918,0.206022,0.485908
30103,AMED,2017-07-12,58.970001,-0.002406,0.152455,0.160072,0.689752
30104,AMED,2017-07-26,48.689999,-0.120590,0.079048,0.075266,0.677474
30105,AMED,2019-03-13,122.250000,-0.113183,-0.090947,0.683718,0.751574


Dodanie kolumn z cenami ropy brent oraz euro i jena w dolarach

In [30]:
oej_data_df = pd.read_excel("https://raw.githubusercontent.com/Aspartamei/Projekt2022_UMwF/main/Dane_OEJv2.xlsx")

oej_data_df["Date"] = pd.to_datetime(oej_data_df["Date"])

full_data_df = pd.merge(mean_ror_df, oej_data_df, how="left", on=["Date"])
full_data_df

,Comp,Date,Close,month_ROR,qtr_ROR,year_ROR,Score,Oil,EUR,JPY
0,UBA,2004-03-24,6.163362,0.083411,NaN,NaN,0.741122,33.01,0.82,106.23
1,UBA,2005-06-15,7.158419,0.087692,0.098211,0.254842,0.803913,54.50,0.83,109.19
2,UBA,2005-06-29,7.281594,0.045381,0.134050,0.223035,0.806620,56.15,0.83,110.41
3,UBA,2005-07-13,7.605220,0.066313,0.228741,0.219617,0.923381,58.27,0.83,111.86
4,UBA,2005-07-27,7.613519,0.062501,0.249711,0.305484,0.923270,58.01,0.83,112.39
...,...,...,...,...,...,...,...,...,...,...
30102,AMED,2010-02-10,56.180000,0.054901,0.305918,0.206022,0.485908,72.54,0.73,89.92
30103,AMED,2017-07-12,58.970001,-0.002406,0.152455,0.160072,0.689752,47.74,0.88,113.16
30104,AMED,2017-07-26,48.689999,-0.120590,0.079048,0.075266,0.677474,50.97,0.85,111.17
30105,AMED,2019-03-13,122.250000,-0.113183,-0.090947,0.683718,0.751574,67.55,0.88,111.17


Oczyszczenie zbioru z braków danych w kolumnach Close, Oil, EUR i JPY.

In [31]:
no_close_data = []
for close in full_data_df.Close.unique():
    close_df = full_data_df[full_data_df.Close == close]
    if close_df.Score.isna().sum() == len(close_df.Score):
        no_close_data.append(close)
no_oil_data = []
for oil in full_data_df.Oil.unique():
    oil_df = full_data_df[full_data_df.Oil == oil]
    if oil_df.Score.isna().sum() == len(oil_df.Score):
        no_oil_data.append(oil)
no_eur_data = []
for eur in full_data_df.EUR.unique():
    eur_df = full_data_df[full_data_df.EUR == eur]
    if eur_df.Score.isna().sum() == len(eur_df.Score):
        no_eur_data.append(eur)
no_jpy_data = []
for jpy in full_data_df.JPY.unique():
    jpy_df = full_data_df[full_data_df.JPY == jpy]
    if jpy_df.Score.isna().sum() == len(jpy_df.Score):
        no_jpy_data.append(jpy)

In [32]:
data_final = pd.DataFrame()
data_final = full_data_df[~full_data_df['Close'].isin(no_close_data)]
data_final = full_data_df[~full_data_df['Oil'].isin(no_oil_data)]
data_final = full_data_df[~full_data_df['EUR'].isin(no_eur_data)]
data_final = full_data_df[~full_data_df['JPY'].isin(no_jpy_data)]

In [33]:
data_final

,Comp,Date,Close,month_ROR,qtr_ROR,year_ROR,Score,Oil,EUR,JPY
0,UBA,2004-03-24,6.163362,0.083411,NaN,NaN,0.741122,33.01,0.82,106.23
1,UBA,2005-06-15,7.158419,0.087692,0.098211,0.254842,0.803913,54.50,0.83,109.19
2,UBA,2005-06-29,7.281594,0.045381,0.134050,0.223035,0.806620,56.15,0.83,110.41
3,UBA,2005-07-13,7.605220,0.066313,0.228741,0.219617,0.923381,58.27,0.83,111.86
4,UBA,2005-07-27,7.613519,0.062501,0.249711,0.305484,0.923270,58.01,0.83,112.39
...,...,...,...,...,...,...,...,...,...,...
30102,AMED,2010-02-10,56.180000,0.054901,0.305918,0.206022,0.485908,72.54,0.73,89.92
30103,AMED,2017-07-12,58.970001,-0.002406,0.152455,0.160072,0.689752,47.74,0.88,113.16
30104,AMED,2017-07-26,48.689999,-0.120590,0.079048,0.075266,0.677474,50.97,0.85,111.17
30105,AMED,2019-03-13,122.250000,-0.113183,-0.090947,0.683718,0.751574,67.55,0.88,111.17


In [34]:
data_final.to_csv("data_final.csv")